[View in Colaboratory](https://colab.research.google.com/github/rutgerhofste/notebook_gis_plotting/blob/master/03_folium_popups.ipynb)

Author: Rutger Hofste  
Date: 2018 07 02  

## Popups


this is a nice first step but you want to inspect the underlying data. This involves a couple more steps and requires a per feature approach which is much more computationally demanding. The process can be paralellized though. 

In [0]:
!pip install folium branca geojson geopandas --quiet

In [0]:
import folium
import branca
import geojson
import pandas as pd
import geopandas as gpd

In [0]:
# load a sample geodataframe
gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [0]:
def pre_process_gdf(gdf_in):    
    """Assert the crs is set to 4326 and geometry column name is 'geometry'
    """
    gdf = gdf_in.copy()
    gdf = gdf.to_crs(epsg='4326')
    gdf = gdf.rename(columns={gdf.geometry.name:"geometry"})
    return gdf    
  
def row_geojson_feature(row):    
    geojson_feature = geojson.Feature(geometry=row["geometry"], properties={})
    return geojson_feature

def row_popup(row):
    width, height = 200,200
    attributes = pd.DataFrame(row.drop("geometry"))
    html_table = attributes.to_html()
    i_frame = branca.element.IFrame(html_table, width=width, height=height)
    return i_frame

def zip_features_popups(row):
    folium_feature = folium.features.GeoJson(row["gjson"])
    folium_feature.add_child(folium.Popup(row["i_frame"]))
    return folium_feature

In [0]:
gdf_temp = gdf.copy()

In [0]:
# Generate geosjon representations of geometries
gdf_temp["gjson"] = gdf_temp.apply(row_geojson_feature,axis=1)

In [0]:
# Create iframes as popups. 
gdf_temp["i_frame"] = gdf.apply(row_popup,1)

In [0]:
# Zip geosjon features and popups in folium features (This step is very slow, 
# todo: look into speed ups.)
gdf_temp["features"] = gdf_temp.apply(zip_features_popups,axis=1)

In [0]:
# Create folium feature group
feature_group_1 = folium.FeatureGroup(name="test")

In [0]:
features_list = gdf_temp["features"].tolist()

for feature in features_list:
    feature.add_to(feature_group_1)

In [12]:
m = folium.Map(location = [52,4],
               zoom_start=6)
m.add_child(feature_group_1)
m

In [0]:
# Everything in one function fro later use. 

def gdf_to_feature_group(gdf,name="noname"):
    """ 
    Converts a geodataframe into a folium featuregroup.
    
    Usage: you can add the folium feature group to a map by
    
    m = folium.Map(location = [52,4],
                   zoom_start=6)
    m.add_child(feature_group)
    m.add_child(folium.LayerControl())
    
    
    Args:
        gdf (gpd.GeoDataFrame) : Geodataframe.
        name (string) : Name for folium group.
    
    Returns:
        feature_group (folium feature group): folium feature group.
    
    """
    
    gdf_clean = pre_process_gdf(gdf)
    gdf_temp = gdf_clean.copy()
    # Generate geosjon representations of geometries
    gdf_temp["gjson"] = gdf_clean.apply(row_geojson_feature,1)
    gdf_temp["i_frame"] = gdf_clean.apply(row_popup,1)
    gdf_temp["features"] = gdf_temp.apply(zip_features_popups,axis=1)
    
    # Add folium features to folium group
    feature_group = folium.FeatureGroup(name=name)
    features_list = gdf_temp["features"].tolist()
    for feature in features_list:
        feature.add_to(feature_group)
    return feature_group

In [0]:
feature_group = gdf_to_feature_group(gdf,name="countries")

In [18]:
try:
    del(m3)
except:
    pass
m3 = folium.Map(location = [52,4],
               zoom_start=6)
m3.add_child(feature_group)
m3